In [1]:
!pip install pettingzoo


   ---------------------------------------- 0.0/847.8 kB ? eta -:--:--
   - -------------------------------------- 30.7/847.8 kB 1.3 MB/s eta 0:00:01
   --- ----------------------------------- 81.9/847.8 kB 919.0 kB/s eta 0:00:01
   ----- -------------------------------- 112.6/847.8 kB 939.4 kB/s eta 0:00:01
   ---------- ----------------------------- 225.3/847.8 kB 1.4 MB/s eta 0:00:01
   ------------------- -------------------- 409.6/847.8 kB 2.0 MB/s eta 0:00:01
   -------------------------------- ------- 686.1/847.8 kB 2.5 MB/s eta 0:00:01
   ---------------------------------------- 847.8/847.8 kB 3.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: C:\Users\ilhui\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import random
from pettingzoo.mpe import simple_tag_v3
from scipy.special import softmax

# Inicializar el entorno y configuraciones
env = simple_tag_v3.env(render_mode="human")
env.reset(seed=42)

# Parámetros de la política
learning_rate = 0.01
discount_factor = 0.9
action_space = 5  # 5 posibles acciones en simple_tag_v3 (parada, izquierda, derecha, adelante, atrás)
epsilon = 0.5  # Para epsilon-greedy

# Definir el total de estados como combinación de posiciones y direcciones posibles (ejemplo simplificado)
total_states = (env.observation_space(env.agents[0]).shape[0], action_space)
model_params = np.zeros((total_states[0], total_states[1]))

# Funciones de ayuda
def calc_state(observation):
    """Calcula el estado basado en la observación (posición y dirección)"""
    return int(observation.sum()) % total_states[0]

def get_action(state, exploration=True):
    """Obtiene una acción usando una política epsilon-greedy"""
    if exploration and random.uniform(0, 1) < epsilon:
        return np.random.choice(action_space)
    else:
        return model_params[state].argmax()

def train(experience, model_params, lr=learning_rate, df=discount_factor):
    """Entrena el modelo actualizando parámetros usando las recompensas"""
    rtgs = 0  # Suma de recompensas futuras con descuento
    for prev_state, action_taken, state, reward, final in reversed(experience):
        rtgs = rtgs * df + reward
        model_params[prev_state][action_taken] += lr * (rtgs - model_params[prev_state][action_taken])

# Bucle de episodios
num_episodes = 10
for episode in range(num_episodes):
    env.reset()
    experience = {agent: [] for agent in env.agents}
    terminated, truncated = {agent: False for agent in env.agents}, {agent: False for agent in env.agents}
    
    print(f"\nInicio del Episodio {episode + 1}")

    # Verificar si todos los agentes han terminado para evitar bucle infinito
    while True:
        all_terminated = True
        for agent in env.agent_iter():
            observation, reward, termination, truncation, info = env.last()
            
            # Comprobar si el agente ha terminado
            if termination or truncation:
                action = None
                print(f"{agent} ha terminado y no toma acción.")
            else:
                # Calcular el estado y tomar acción
                state = calc_state(observation)
                action = get_action(state)  # Acción calculada según la política
                print(f"{agent} toma acción {action} con recompensa {reward}")
                
                # Guardar experiencia para entrenamiento
                next_observation, next_reward, next_termination, next_truncation, _ = env.last()
                next_state = calc_state(next_observation)
                
                # Registrar experiencia del agente actual
                experience[agent].append([state, action, next_state, reward, termination or truncation])
                
                # Entrenar el modelo del agente
                train(experience[agent], model_params)

            # Toma la acción en el entorno solo si no ha terminado
            env.step(action)
            
            # Actualizar estado de terminación y truncación
            terminated[agent] = termination
            truncated[agent] = truncation

            # Verificar si el agente aún no ha terminado
            if not (termination or truncation):
                all_terminated = False  # Si algún agente no ha terminado, el episodio continúa
        
        # Si todos los agentes han terminado, salir del bucle
        if all_terminated:
            break

    print(f"Fin del Episodio {episode + 1}")

env.close()


pygame 2.6.1 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html

Inicio del Episodio 1
adversary_0 toma acción 0 con recompensa 0.0
adversary_1 toma acción 0 con recompensa 0.0
adversary_2 toma acción 0 con recompensa 0.0
agent_0 toma acción 0 con recompensa 0.0
adversary_0 toma acción 4 con recompensa 0.0
adversary_1 toma acción 0 con recompensa 0.0
adversary_2 toma acción 0 con recompensa 0.0
agent_0 toma acción 0 con recompensa 0.0
adversary_0 toma acción 3 con recompensa 0.0
adversary_1 toma acción 0 con recompensa 0.0
adversary_2 toma acción 0 con recompensa 0.0
agent_0 toma acción 0 con recompensa 0.0
adversary_0 toma acción 4 con recompensa 0.0
adversary_1 toma acción 0 con recompensa 0.0
adversary_2 toma acción 2 con recompensa 0.0
agent_0 toma acción 0 con recompensa 0.0
adversary_0 toma acción 0 con recompensa 0.0
adversary_1 toma acción 0 con recompensa 0.0
adversary_2 toma acción 0 con recompensa 0.0
agent_0 toma acción 0 con